In [5]:
import numpy as np
import sys

# This class is to read grammar rules from the file and make the grammar 

class GrammarReader:
    # initialize the parser by adding PCF grammar rules
    # logarithm of the probabilities have been given in this file
    def __init__ (self, rules='data/weighted.rule'):
        self.grammar = self.readGrammar(rules)

    # Given a file containing weighted rules, grammarFile, return a dictionary of
    # those rules.
    def readGrammar (self, grammarFile):
        grammar = {}
        rules = open(grammarFile, 'r')
    
        for rule in rules:
            tmp = rule.split()
            lhs = tmp[0]
            rhs = ' '.join(tmp[1:-1])
            weight = float(tmp[-1])

            if lhs in grammar:
                grammar[lhs][rhs] = weight
            else:
                grammar[lhs] = {rhs: weight}

        rules.close()

        return grammar
    
# This class is the basic building block for creating CYK chart 

class ChartElement:
    def __init__ (self, *args, **kwargs):
        self.ruleList = kwargs.get('ruleList', None)
        
        if self.ruleList is None:
            
            self.grammar = kwargs.get('grammar')
            self.ruleList = RuleList()
        
            if (kwargs.get('isLexicon')):
                
                self.ruleList = self.initTerminal(kwargs.get('element1'), kwargs.get('index1'))
            else :
                self.ruleList = self.initNonterminal(kwargs.get('element1'), kwargs.get('element2'),kwargs.get('index1'), kwargs.get('index2'))
        
    def initTerminal (self, input, index):
        #search through grammar[:][input]
        # get all matching lhs, weights
        #for all matching rules, create rules, origin = '0 0'
        
        lexiconRules = self.findMatchingRules(input, True, 0.0, str(index)+' '+ str(index))
        # then look again for lhs of these rules and check whether they can be unary rules
        # add new unary rules also to the list
        for lexiconRule in lexiconRules:
            matchingUnaryRules = self.findMatchingRules(lexiconRule.lhs, False, lexiconRule.weight, lexiconRule.origin)
            lexiconRules.extend(matchingUnaryRules)
        return lexiconRules
        
    def initNonterminal (self, lhsRules, rhsRules, index1, index2):
        nonTerminalRules = RuleList()
        # get lhs.lhs list and rhs.lhs list
        # look for binary rules which can be made from those
        for lhs_lhs in lhsRules:
            for rhs_lhs in rhsRules:
                binary_rhs = lhs_lhs.lhs + ' ' + rhs_lhs.lhs
                
                nonTerminalRules.extend(self.findMatchingRules(binary_rhs, False, lhs_lhs.weight + rhs_lhs.weight, index1 + ',' + index2 ))
        # then look again for lhs of these rules and check whether they can be unary rules
        
        for nonTerminalRule in nonTerminalRules:
            matchingUnaryRules = self.findMatchingRules(nonTerminalRule.lhs, False, nonTerminalRule.weight, nonTerminalRule.origin)
            nonTerminalRules.extend(matchingUnaryRules)
        # add new unary rules also to the list
        
        return nonTerminalRules
        
    def findMatchingRules (self, rhs, isLexicon, prob, origin):
        
        resultRules = RuleList()
        for (lhs, d) in self.grammar.items():
            for current_rhs in d:
                if current_rhs == rhs:
                    rule = Rule(lhs, rhs, prob + d[current_rhs],origin, isLexicon)
                    resultRules.append(rule)

        # Handle unseen words
        if isLexicon and not resultRules:
            print('Unknown word is found : ' + rhs)
            sys.exit()

        return resultRules

# This class is inherited from default Python list in order to override append and extend methods
# These methods were extended as we only need to keep the highest probabilistic among language rules with same LHS
    
class RuleList(list):
    def append(self, newRule):
        isNew = True
        for currentRule in self:
            if currentRule.lhs == newRule.lhs:
                isNew = False
                if currentRule.weight < newRule.weight:
                    currentRule.rhs = newRule.rhs
                    currentRule.weight = newRule.weight
                    currentRule.origin = newRule.origin
                    currentRule.isLexicon = newRule.isLexicon
                break
        if isNew:
            super(RuleList, self).append(newRule)        
        
    def extend(self, newRules):
        for newRule in newRules:
            self.append(newRule)
    
# This class is to store details of all the rules separately
class Rule:
    def __init__(self, lhs, rhs, weight, origin, isLexicon):
        self.lhs = lhs
        self.rhs = rhs
        self.weight = weight
        self.origin = origin
        self.isLexicon = isLexicon
    
# CYK Algorithm

class CYKAlgo:
    def __init__ (self, input, grammar):
        self.input = input.split()
        inputSize = len(self.input)
        self.grammar = grammar
        self.chart = np.empty(shape=(inputSize,inputSize), dtype=object)
        self.parse(0, inputSize-1)
     
    # A dynamic program based parsing logic
    # if index1 == index2, chart[index1][index2] = logic for parsing lexicon
    # else chart[index1][index2] = max(index1<= k < index2)[parse(index1,k)+parse(k+1,index2)]
    def parse (self, index1, index2):
        if self.chart[index1,index2] is not None and len(self.chart[index1,index2].ruleList) != 0 :
            return self.chart[index1,index2]
        if index1 == index2:
            self.chart[index1,index1] = ChartElement(index1=index1, element1=self.input[index1], grammar=self.grammar, isLexicon=True)
            
            return self.chart[index1,index1]
        else :
            maxProbRules = RuleList()
            
            for k in range(index1,index2):
                
                lhs = self.parse(index1, k).ruleList
                
                rhs = self.parse(k+1, index2).ruleList
                
                if(len(lhs) != 0 and len(rhs) != 0) :
                    
                    currentRules = ChartElement(index1=str(index1) +' '+ str(k), index2=str(k+1)+' '+ str(index2), element1=lhs, element2=rhs, grammar=self.grammar, isLexicon=False).ruleList
                
                    for rule in currentRules:
                        maxProbRules.append(rule)
                    
            self.chart[index1,index2] = ChartElement(ruleList=maxProbRules)
            return self.chart[index1,index2]

#=================== Enf of class definitions ========================================       

# Main method of the program
def main():
    defaultsInputSentence = 'good father gone for Market place'
    inputSentence =''
    #Get user input 
    print ('PCFGParser\n===========')
    print('Sample sentences can be found in data/samples.txt \n')
    userInputSent = input('Type input sentence : ')
    if len(userInputSent) > 0:
        inputSentence = userInputSent
    else:
        inputSentence = defaultsInputSentence
    
    print('Parsed results for input : ' + inputSentence + '\n')
    parseTreeChart = CYKAlgo(inputSentence, GrammarReader().grammar).chart
    printParseTree(parseTreeChart)
        
def printParseTree (chart):
    inputLen = chart.shape[0]
    
    #print parse tree
    # look at the rules in chart[0][inputLen-1]
    # if it is not empty, get the rule with the highest probability
    #look at its origin and recursively print the rules until you meet origin x,x
    #if such thing seen, check whether you can expand to more unary rules in same element
    # else finish
    
    rootElement = chart[0][inputLen-1]
    
    if rootElement is not None and len(rootElement.ruleList) != 0:
        maxProbRootRule = None
        maxProb = float('-inf')
        for rule in rootElement.ruleList:
            if rule.weight > maxProb:
                maxProb = rule.weight
                maxProbRootRule = rule
        print('Parse Tree\n===========')
        printRule(maxProbRootRule, chart)
        print('\n')
        print('CYK Chart \n==========')
        printCYKChart(chart)
            
        
    else :
        print('No complete parse tree was found! \n Following is the final CYK chart')
        printCYKChart(chart)
                            
def printRule(rule, chart):
    # look for origins of current element's rhs
    originRules = None
    if not rule.isLexicon:
        originRules = getOriginRules(rule, chart)
    print('( ' + rule.lhs, sep=' ', end='', flush=True)
    if originRules:
        for origin in originRules:
            printRule(origin, chart) 
    else :
        print(' ' + rule.rhs, sep=' ', end='', flush=True)
    print(' )', sep=' ', end='', flush=True)
    
def getOriginRules (rule, chart):
    rhsElements = rule.rhs.split()
    originrhs = rule.origin.split(',')
    originRules = []
    for index in range(0, len(rhsElements)):
        originElementIndex = originrhs[index].split()
        originChartElement = chart[int(originElementIndex[0])][int(originElementIndex[1])]
        
        for rule in originChartElement.ruleList:
            if (rule.lhs == rhsElements[index]):
                originRules.append(rule)
    return originRules     

def printCYKChart(chart):
    inputLen = chart.shape[0]
    for i in range (0, inputLen):
            for j in range (0, inputLen):
                if chart[i][j] is not None:
                    print('chart index:'+str(i)+str(j)+' ')
                    if len(chart[i][j].ruleList) != 0:
                        for rule in chart[i][j].ruleList:
                            print('\t' + rule.lhs + '->' + rule.rhs + ' weight:' + str(rule.weight) + ' indecies of rhs origin :'+rule.origin)
    
if __name__ == '__main__':
    main()

PCFGParser
Sample sentences can be found in data/samples.txt 

Type input sentence : Each student came to the Hospital
Parsed results for input : Each student came to the Hospital

Parse Tree
( S( NP( DT Each )( NN student ) )( VP( VBD came )( PP( IN to )( NP( DT the )( NN Hospital ) ) ) ) )

CYK Chart 
chart index:00 
	DT->Each weight:-4.54965747606 indecies of rhs origin :0 0
chart index:01 
	NP->DT NN weight:-11.55182322953 indecies of rhs origin :0 0,1 1
chart index:02 
chart index:03 
chart index:04 
chart index:05 
	NP->NP VP weight:-34.248606242653395 indecies of rhs origin :0 1,2 5
	FRAG->NP VP weight:-30.477123260003395 indecies of rhs origin :0 1,2 5
	S->NP VP weight:-28.285393201981034 indecies of rhs origin :0 1,2 5
chart index:11 
	NN->student weight:-5.71867108715 indecies of rhs origin :1 1
chart index:12 
chart index:13 
chart index:14 
chart index:15 
chart index:22 
	VBD->came weight:-4.1666652238 indecies of rhs origin :2 2
chart index:23 
chart index:24 
chart index